# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [5]:
import numpy as np
import os
#from scipy.misc import imread, imresize
from imageio import imread
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [6]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
from keras.regularizers import l2
import tensorflow as tf
tf.set_random_seed(30)

In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [9]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 15 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [10]:
default_y = 120
default_z = 120
image_idx = list(range(0,30,1))

In [11]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = image_idx #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),default_y,default_z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = cv2.resize(image,(default_y,default_z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0])-np.min(image[:, :, 0])) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1])-np.min(image[:, :, 1])) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2])-np.min(image[:, :, 2])) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        
        remaining_batch = len(folder_list) - (num_batches * batch_size)      
        batch_data = np.zeros((remaining_batch,len(img_idx),default_y,default_z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_batch,5))
        for folder in range(remaining_batch): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = cv2.resize(image,(default_y,default_z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0])-np.min(image[:, :, 0]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1])-np.min(image[:, :, 1])) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2])-np.min(image[:, :, 2])) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*remaining_batch)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels                                                              


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [12]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # REducelronplateau code
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

# Model

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

### Model 1

- No. Convolution Layers - 6
- Convolution Filter Size- (2,2,2)
- Pooling filter size - (2,2,2)
- Batch Size - 15
- Epochs - 30
- Image size - (120, 120)

In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers



model = Sequential()
model.add(Conv3D(8, (2, 2, 2), padding='same',
                 input_shape=(len(image_idx), default_z, default_y, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(16, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))

model.add(Conv3D(16, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(5))
model.add(Activation('softmax'))

In [15]:
optimiser = optimizers.Adam(lr = 0.001) #optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 30, 120, 120, 8)   200       
_________________________________________________________________
activation_10 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 29, 119, 119, 16)  1040      
_________________________________________________________________
activation_11 (Activation)   (None, 29, 119, 119, 16)  0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 29, 119, 119, 16)  64        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 14, 59, 59, 16)    0         
__________

In [17]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
45/45 [==============================] - 68s 2s/step - loss: 1.3496 - categorical_accuracy: 0.5171 - val_loss: 1.4106 - val_categorical_accuracy: 0.5600

Epoch 00001: saving model to model_init_2020-05-2505_42_06.953425/model-00001-1.32584-0.52036-1.41058-0.56000.h5
Epoch 2/20
45/45 [==============================] - 64s 1s/step - loss: 1.2698 - categorical_accuracy: 0.5438 - val_loss: 1.2639 - val_categorical_accuracy: 0.6300

Epoch 00002: saving model to model_init_2020-05-2505_42_06.953425/model-00002-1.26252-0.54751-1.26392-0.63000.h5
Epoch 3/20
45/45 [==============================] - 65s 1s/step - loss: 1.1973 - categorical_accuracy: 0.5721 - val_loss: 1.4629 - val_categorical_accuracy: 0.5700

Epoch 00003: saving model to model_init_2020-05-2505_42_06.953425/model-00003-1.18852-0.58220-1.46293-0.57000.h5
Epoch 4/20
45/45 [==============================] - 65s 1s/step - loss: 1.3096 - categorical_accuracy: 0.5676 - val_loss: 1.4486 - val_categorical_accuracy: 0.5200

E

`Running for 10 more epochs`

In [18]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
45/45 [==============================] - 68s 2s/step - loss: 0.7445 - categorical_accuracy: 0.7633 - val_loss: 1.2240 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-05-2505_42_06.953425/model-00001-0.62487-0.77677-1.22396-0.60000.h5
Epoch 2/10
45/45 [==============================] - 64s 1s/step - loss: 0.7609 - categorical_accuracy: 0.7396 - val_loss: 1.2406 - val_categorical_accuracy: 0.6700

Epoch 00002: saving model to model_init_2020-05-2505_42_06.953425/model-00002-0.62585-0.75264-1.24061-0.67000.h5
Epoch 3/10
45/45 [==============================] - 64s 1s/step - loss: 0.6879 - categorical_accuracy: 0.7587 - val_loss: 1.4294 - val_categorical_accuracy: 0.6400

Epoch 00003: saving model to model_init_2020-05-2505_42_06.953425/model-00003-0.61851-0.76621-1.42942-0.64000.h5
Epoch 4/10
45/45 [==============================] - 63s 1s/step - loss: 0.6470 - categorical_accuracy: 0.7763 - val_loss: 1.2410 - val_categorical_accuracy: 0.6500

E

 - loss: 0.6148 
 - categorical_accuracy: 0.7587 
 - val_loss: 1.5143 
 - val_categorical_accuracy: 0.6100

### Model 2

- No. Convolution Layers - 6
- Convolution Filter Size- (2,2,2)
- Pooling filter size - (2,2,2)
- Batch Size - 15
- Epochs - 25
- Image size - (120, 120)

In [25]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers



model = Sequential()
model.add(Conv3D(8, (2, 2, 2), padding='same',
                 input_shape=(len(image_idx), default_z, default_y, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(16, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(16, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(5))
model.add(Activation('softmax'))

In [26]:
optimiser = optimizers.Adam(lr = 0.001) #optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 120, 120, 8)   200       
_________________________________________________________________
activation_28 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 29, 119, 119, 16)  1040      
_________________________________________________________________
activation_29 (Activation)   (None, 29, 119, 119, 16)  0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 29, 119, 119, 16)  64        
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 14, 59, 59, 16)    0         
__________

In [27]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/25
45/45 [==============================] - 69s 2s/step - loss: 11.4648 - categorical_accuracy: 0.2208 - val_loss: 8.1885 - val_categorical_accuracy: 0.4200

Epoch 00001: saving model to model_init_2020-05-2505_42_06.953425/model-00001-11.53111-0.22474-8.18849-0.42000.h5
Epoch 2/25
45/45 [==============================] - 60s 1s/step - loss: 9.7442 - categorical_accuracy: 0.2742 - val_loss: 8.8353 - val_categorical_accuracy: 0.3200

Epoch 00002: saving model to model_init_2020-05-2505_42_06.953425/model-00002-9.70566-0.27903-8.83532-0.32000.h5
Epoch 3/25
45/45 [==============================] - 64s 1s/step - loss: 8.5166 - categorical_accuracy: 0.3230 - val_loss: 8.1728 - val_categorical_accuracy: 0.3700

Epoch 00003: saving model to model_init_2020-05-2505_42_06.953425/model-00003-8.52102-0.32278-8.17281-0.37000.h5
Epoch 4/25
45/45 [==============================] - 64s 1s/step - loss: 6.2760 - categorical_accuracy: 0.4162 - val_loss: 2.1863 - val_categorical_accuracy: 0.6000


- loss: 0.4377 
- categorical_accuracy: 0.8282
- val_loss: 1.0525 
- val_categorical_accuracy: 0.6500

### Model 3

- No. Convolution Layers - 6
- Convolution Filter Size- (2,2,2)
- Pooling filter size - (2,2,2)
- Batch Size - 15
- Epochs - 25
- Image size - (120, 120)

In [112]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers



model = Sequential()
model.add(Conv3D(8, (2, 2, 2), padding='same',
                 input_shape=(len(image_idx), default_z, default_y, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(16, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.1))

model.add(Conv3D(16, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(5))
model.add(Activation('softmax'))

In [116]:
optimiser = optimizers.Adam(lr = 0.001) # optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_147 (Conv3D)          (None, 30, 120, 120, 8)   200       
_________________________________________________________________
activation_216 (Activation)  (None, 30, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_149 (Bat (None, 30, 120, 120, 8)   32        
_________________________________________________________________
conv3d_148 (Conv3D)          (None, 29, 119, 119, 16)  1040      
_________________________________________________________________
activation_217 (Activation)  (None, 29, 119, 119, 16)  0         
_________________________________________________________________
batch_normalization_150 (Bat (None, 29, 119, 119, 16)  64        
_________________________________________________________________
max_pooling3d_74 (MaxPooling (None, 14, 59, 59, 16)    0         
__________

In [117]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 15
Source path =  ./Project_data/train ; batch size = 15
Epoch 1/25
45/45 [==============================] - 78s 2s/step - loss: 1.9967 - categorical_accuracy: 0.3483 - val_loss: 1.9374 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2020-05-2509_35_02.469855/model-00001-1.97679-0.35445-1.93743-0.46000.h5
Epoch 2/25
45/45 [==============================] - 51s 1s/step - loss: 1.5884 - categorical_accuracy: 0.4890 - val_loss: 1.4103 - val_categorical_accuracy: 0.5700

Epoch 00002: saving model to model_init_2020-05-2509_35_02.469855/model-00002-1.52214-0.49170-1.41029-0.57000.h5
Epoch 3/25
45/45 [==============================] - 66s 1s/step - loss: 1.4507 - categorical_accuracy: 0.4965 - val_loss: 1.6549 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-05-2509_35_02.469855/model-00003-1.40006-0.50528-1.65488-0.53000.h5
Epoch 4/25
45/45 [==============================] - 65s 1s/s

 - loss: 0.7514 
 - categorical_accuracy: 0.7455
 - val_loss: 0.9331 
 - val_categorical_accuracy: 0.7600

### Model 4

- No. Convolution Layers - 6
- Convolution Filter Size- (2,2,2)
- Pooling filter size - (2,2,2)
- Batch Size - 15
- Epochs - 20
- Image size - (120, 120)

In [69]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers



model = Sequential()
model.add(Conv3D(16, (2, 2, 2), padding='same',
                 input_shape=(len(image_idx), default_z, default_y, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))

model.add(Conv3D(16, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (2, 2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.05))


model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(5))
model.add(Activation('softmax'))

In [70]:
optimiser = optimizers.Adam(lr = 0.001) #optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_89 (Conv3D)           (None, 30, 120, 120, 16)  400       
_________________________________________________________________
activation_104 (Activation)  (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_113 (Bat (None, 30, 120, 120, 16)  64        
_________________________________________________________________
conv3d_90 (Conv3D)           (None, 29, 119, 119, 32)  4128      
_________________________________________________________________
activation_105 (Activation)  (None, 29, 119, 119, 32)  0         
_________________________________________________________________
batch_normalization_114 (Bat (None, 29, 119, 119, 32)  128       
_________________________________________________________________
max_pooling3d_45 (MaxPooling (None, 14, 59, 59, 32)    0         
__________

In [72]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
45/45 [==============================] - 72s 2s/step - loss: 0.8393 - categorical_accuracy: 0.6669 - val_loss: 1.4640 - val_categorical_accuracy: 0.6100

Epoch 00001: saving model to model_init_2020-05-2511_53_47.602867/model-00001-0.81794-0.67873-1.46405-0.61000.h5
Epoch 2/10
45/45 [==============================] - 67s 1s/step - loss: 0.8269 - categorical_accuracy: 0.7083 - val_loss: 7.6044 - val_categorical_accuracy: 0.2700

Epoch 00002: saving model to model_init_2020-05-2511_53_47.602867/model-00002-0.80945-0.71493-7.60443-0.27000.h5
Epoch 3/10
45/45 [==============================] - 66s 1s/step - loss: 0.9134 - categorical_accuracy: 0.6920 - val_loss: 1.6238 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_init_2020-05-2511_53_47.602867/model-00003-0.85509-0.69834-1.62377-0.54000.h5
Epoch 4/10
45/45 [==============================] - 66s 1s/step - loss: 0.8673 - categorical_accuracy: 0.6892 - val_loss: 1.5191 - val_categorical_accuracy: 0.6200

E

In [73]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5
45/45 [==============================] - 71s 2s/step - loss: 0.3662 - categorical_accuracy: 0.8476 - val_loss: 0.9538 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-05-2511_53_47.602867/model-00001-0.34017-0.85671-0.95382-0.65000.h5
Epoch 2/5
45/45 [==============================] - 66s 1s/step - loss: 0.4742 - categorical_accuracy: 0.8432 - val_loss: 1.0509 - val_categorical_accuracy: 0.6700

Epoch 00002: saving model to model_init_2020-05-2511_53_47.602867/model-00002-0.40025-0.85219-1.05091-0.67000.h5
Epoch 3/5
45/45 [==============================] - 66s 1s/step - loss: 0.4757 - categorical_accuracy: 0.8270 - val_loss: 0.9717 - val_categorical_accuracy: 0.7200

Epoch 00003: saving model to model_init_2020-05-2511_53_47.602867/model-00003-0.42645-0.84163-0.97168-0.72000.h5
Epoch 4/5
45/45 [==============================] - 67s 1s/step - loss: 0.4724 - categorical_accuracy: 0.8285 - val_loss: 1.0156 - val_categorical_accuracy: 0.7500

Epoch

In [74]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5
45/45 [==============================] - 72s 2s/step - loss: 0.4386 - categorical_accuracy: 0.8564 - val_loss: 0.9639 - val_categorical_accuracy: 0.7300

Epoch 00001: saving model to model_init_2020-05-2511_53_47.602867/model-00001-0.39818-0.85973-0.96387-0.73000.h5
Epoch 2/5
45/45 [==============================] - 67s 1s/step - loss: 0.4803 - categorical_accuracy: 0.8566 - val_loss: 1.0230 - val_categorical_accuracy: 0.6900

Epoch 00002: saving model to model_init_2020-05-2511_53_47.602867/model-00002-0.39476-0.87179-1.02300-0.69000.h5
Epoch 3/5
45/45 [==============================] - 66s 1s/step - loss: 0.4365 - categorical_accuracy: 0.8552 - val_loss: 0.7503 - val_categorical_accuracy: 0.8100

Epoch 00003: saving model to model_init_2020-05-2511_53_47.602867/model-00003-0.35396-0.87029-0.75031-0.81000.h5
Epoch 4/5
45/45 [==============================] - 66s 1s/step - loss: 0.4446 - categorical_accuracy: 0.8552 - val_loss: 0.8692 - val_categorical_accuracy: 0.7300

Epoch

- loss: 0.4365 
- categorical_accuracy: 0.8552 
- val_loss: 0.7503 
- val_categorical_accuracy: 0.8100